In [14]:
import requests
import pandas as pd
import json
import time

def search_recording(artist_name, release_title):
    base_url = "https://musicbrainz.org/ws/2/release/"
    params = {
        "query": f"artist:\"{artist_name}\" AND release:\"{release_title}\"",
        "fmt": "json"
    }
    headers = {"User-Agent": "Mozilla/5.0 (bittah.pupil@proton.me)"} # Replace with your app info

    response = requests.get(base_url, params=params, headers=headers)
    response.raise_for_status() # Raise an exception for bad status codes
    data = response.json()
    
    if data and data.get('releases'):
        # Return the MBID of the first matching recording
        return data['releases'][1]['id']
    return None

# Example usage:
artist = "Steely Dan"
album = "Aja"
release_mbid = search_recording(artist, album)
print(f"Recording MBID for '{album}' by '{artist}': {release_mbid}")

Recording MBID for 'Aja' by 'Steely Dan': cb3d5f81-a4cf-4789-b3bf-724fe75ef1c8


In [15]:
async def get_credits_for_recording(release_mbid):
    base_url = f"https://musicbrainz.org/ws/2/release/{release_mbid}"
    params = {
        "inc": "artist-credits+labels+recordings+recording-level-rels+work-rels+work-level-rels+artist-rels + release-groups",  # Include artist relationships
        "fmt": "json"
    }
    headers = {"User-Agent": "Mozilla/5.0 (bittah.pupil@proton.me)"}

    response = requests.get(base_url, params=params, headers=headers)
    response.raise_for_status()
    data = response.json()

    credits = []
    
    if data and data.get('artist-credit'):
       for track in data['media'][0]['tracks']:
            for credit in track['recording']['relations']:
                if credit['attributes'] == []:
                    attributes = 'Not Found'
                else:
                    attributes = credit['attributes']
                credits.append((track['number'], track['title'], credit.get('artist', {}).get('name', 'Not Found'), attributes, credit.get('artist', {}).get('id', 'Not Found')))
            for credit in track['artist-credit']:
                credits.append((track['number'], track['title'], credit.get('name', 'Not Found'), credit.get('attributes', 'Not Found'), credit.get('artist', {}).get('id', 'Not Found')))
    if data['relations']:
       for credit in data['relations']:
           trackNum = 'Not Found'
           trackTitle = 'Not Found'
           if credit['attributes'] == [] or not credit['attributes']:
               attributes = 'Not Found'
           else:
               attributes = credit['attributes']
           credits.append((trackNum, trackTitle, credit.get('artist', {}).get('name', 'Not Found'), attributes, credit.get('artist', {}).get('id', 'Not Found')))

    creditNames = []
    creditList = []
    for credit in credits:
        if credit[2] not in creditNames:
            creditNames.append(credit[2])
            creditList.append((credit[2],credit[3], credit[4]))
    print(creditNames)
    print("\n")
    print(creditList)

    artist_mbid = creditList[0][2]
    print(artist_mbid)
    
    return credits

In [16]:
#Example usage:
if release_mbid:
    albumCredits = await get_credits_for_recording(release_mbid)
    #print(albumCredits)
else:
    print(f"Recording for '{album}' by '{artist}' not found.")

time.sleep(2)

# Create the DataFrame
df = pd.DataFrame(albumCredits, columns = ['Track Number', 'Track Title', 'Artist', 'Role', 'Artist_MBID'])

pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', None)


# Display the DataFrame
df

['Tom Scott', 'Roger Nichols', 'Larry Carlton', 'Donald Fagen', 'Victor Feldman', 'Paul Humphrey', 'Chuck Rainey', 'Joe Sample', 'Gary Katz', 'Venetta Fields', 'Clydie King', 'Rebecca Louis', 'Sherlie Matthews', 'Not Found', 'Steely Dan', 'Walter Becker', 'Denny Dias', 'Steve Gadd', 'Michael Omartian', 'Wayne Shorter', 'Bill Schnee', 'Timothy B. Schmit', 'Pete Christlieb', 'Bernard “Pretty” Purdie', 'Lee Ritenour', 'Gary Coleman', 'Jay Graydon', 'Paul Griffin', 'Don Grolnick', 'Steve Kahn', 'Steve Khan', 'Rick Marotta', 'Michael McDonald', 'Ed Greene', 'Dean Parks', 'Jim Keltner', 'OZ Studios', 'Patricia Mitsui', 'Geoff Westen', 'Chuck Findley', 'Jim Horn', 'Dick Hyde', 'Plas Johnson', 'Jackie Kelso', 'Lew McCreary', 'Bill Perkins', '藤井秀樹', 'Dorothy White']


[('Tom Scott', ['horn'], '901434fc-5ec8-41a1-af59-6ded7de8a9d2'), ('Roger Nichols', ['executive'], 'f706ae5b-de2c-4e4a-adc2-a1b9da839d70'), ('Larry Carlton', ['guitar'], 'f1f81989-dfa9-4bd3-805e-dcf3900c43e3'), ('Donald Fagen', ['

,Track Number,Track Title,Artist,Role,Artist_MBID
0,1,Black Cow,Tom Scott,[horn],901434fc-5ec8-41a1-af59-6ded7de8a9d2
1,1,Black Cow,Roger Nichols,[executive],f706ae5b-de2c-4e4a-adc2-a1b9da839d70
2,1,Black Cow,Larry Carlton,[guitar],f1f81989-dfa9-4bd3-805e-dcf3900c43e3
3,1,Black Cow,Donald Fagen,[synthesizer],70047e57-0153-4117-b0fc-a1d2e322e5ef
4,1,Black Cow,Victor Feldman,[Rhodes piano],c648f7ba-cee7-4f04-aaec-03299409682c
5,1,Black Cow,Paul Humphrey,[drums (drum set)],af7c128b-d6cd-4f6d-8c7d-20a46253918c
6,1,Black Cow,Chuck Rainey,[bass guitar],910504cf-866e-4ee6-90ca-d71da15c18ce
7,1,Black Cow,Joe Sample,[clavinet],526a6127-7f4e-4797-8ae9-927a6e29e2d0
8,1,Black Cow,Tom Scott,[tenor saxophone],901434fc-5ec8-41a1-af59-6ded7de8a9d2
9,1,Black Cow,Tom Scott,[horn],901434fc-5ec8-41a1-af59-6ded7de8a9d2
